In [33]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [34]:
df = pd.read_csv('letter-recognition.data',header=None)

In [35]:
train_df = df[:1600]
test_df = df[16000:16000+400]

In [36]:
X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [37]:
from collections import Counter
def counter(arr):
    return Counter(arr).most_common() # 返回出现频率最高的两个数

In [30]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [38]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [39]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        
        outputs = []
        for xi in x_inputs:
            output = net.activate(xi)
            outputs.append(output)
        
        
        px_outputs = softmax(np.array(outputs).reshape(samplesize, number_of_classification), axis=1)
        # the index of maximum in each line
        pred_outputs = np.argmax(px_outputs, axis = 1)
        # replace index to real number, from higher number to lower number!!
        for i in range(number_of_classification):
            index = number_of_classification - i - 1
            pred_outputs[pred_outputs==index] = numbers[index]
        
        real_outputs = np.array(x_outputs).reshape(samplesize,)
        
        acc = np.sum(pred_outputs == real_outputs)/samplesize

        genome.fitness = acc

In [40]:
numbers = [*range(26)] # number 2 > number 1
number_of_classification = len(numbers)


list_y = y.tolist()
digits_indexes = []
for digit in numbers:
    li = [i for i in range(len(list_y)) if list_y[i] == digit]
    digits_indexes.extend(li)

samplesize = X[digits_indexes].shape[0]
x_inputs =[tuple(c) for c in X[digits_indexes].tolist()]
x_outputs = [tuple(c) for c in y[digits_indexes].reshape(samplesize,1).tolist()]

In [41]:
def run(config_file):
    # Load configuration.
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_file,
    )

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # add a stdout reporter to show progress in the terminal
    reporter = neat.StdOutReporter(False)
    p.add_reporter(reporter)
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #checkpointer = neat.Checkpointer(100)
    #p.add_reporter(checkpointer)
    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 3000)

    return [stats, winner]


samplesize = len(x_outputs)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter26")
[stats, winner] = run(config_path)

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)


 ****** Running generation 0 ****** 

Population's average fitness: 0.03863 stdev: 0.01368
Best fitness: 0.09500 - size: (26, 42) - species 14 - id 83
Average adjusted fitness: 0.033
Mean genetic distance 2.735, standard deviation 0.251
Population of 203 members in 44 species
Total extinctions: 0
Generation time: 24.823 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.04323 stdev: 0.01354
Best fitness: 0.09500 - size: (26, 42) - species 14 - id 83
Average adjusted fitness: 0.037
Mean genetic distance 2.803, standard deviation 0.298
Population of 202 members in 44 species
Total extinctions: 0
Generation time: 19.895 sec (22.359 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.04448 stdev: 0.01335
Best fitness: 0.09500 - size: (26, 42) - species 14 - id 83
Average adjusted fitness: 0.039
Mean genetic distance 2.825, standard deviation 0.308
Population of 203 members in 44 species
Total extinctions: 0
Generation time: 19.354 sec (2

Population's average fitness: 0.05657 stdev: 0.01990
Best fitness: 0.11438 - size: (31, 47) - species 29 - id 3087
Average adjusted fitness: 0.038
Mean genetic distance 3.078, standard deviation 0.399
Population of 194 members in 30 species
Total extinctions: 0
Generation time: 18.863 sec (18.780 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.05760 stdev: 0.01989
Best fitness: 0.11937 - size: (30, 49) - species 27 - id 3221
Average adjusted fitness: 0.052
Mean genetic distance 3.089, standard deviation 0.403
Population of 197 members in 29 species
Total extinctions: 0
Generation time: 18.868 sec (18.848 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.05733 stdev: 0.02046
Best fitness: 0.11937 - size: (30, 49) - species 27 - id 3221
Average adjusted fitness: 0.051
Mean genetic distance 3.081, standard deviation 0.400
Population of 194 members in 28 species
Total extinctions: 0
Generation time: 18.710 sec (18.816 average)

Population's average fitness: 0.06315 stdev: 0.02379
Best fitness: 0.13875 - size: (31, 51) - species 27 - id 5020
Average adjusted fitness: 0.050
Mean genetic distance 3.120, standard deviation 0.508
Population of 201 members in 15 species
Total extinctions: 0
Generation time: 20.113 sec (19.822 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.06151 stdev: 0.02423
Best fitness: 0.13875 - size: (31, 51) - species 27 - id 5020
Average adjusted fitness: 0.043
Mean genetic distance 3.136, standard deviation 0.506
Population of 198 members in 14 species
Total extinctions: 0
Generation time: 20.917 sec (19.936 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.06247 stdev: 0.02373
Best fitness: 0.13875 - size: (31, 51) - species 27 - id 5020
Average adjusted fitness: 0.038
Mean genetic distance 3.128, standard deviation 0.511
Population of 203 members in 14 species
Total extinctions: 0
Generation time: 19.987 sec (20.017 average)

Population's average fitness: 0.06558 stdev: 0.02476
Best fitness: 0.13937 - size: (38, 59) - species 46 - id 10164
Average adjusted fitness: 0.047
Mean genetic distance 3.067, standard deviation 0.524
Population of 198 members in 14 species
Total extinctions: 0
Generation time: 21.442 sec (21.123 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.06456 stdev: 0.02512
Best fitness: 0.13937 - size: (38, 59) - species 46 - id 10164
Average adjusted fitness: 0.049
Mean genetic distance 3.070, standard deviation 0.532
Population of 204 members in 14 species
Total extinctions: 0
Generation time: 22.591 sec (21.286 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.06459 stdev: 0.02550
Best fitness: 0.13937 - size: (38, 59) - species 46 - id 10164
Average adjusted fitness: 0.053
Mean genetic distance 3.051, standard deviation 0.514
Population of 203 members in 14 species
Total extinctions: 0
Generation time: 22.001 sec (21.363 avera

Population's average fitness: 0.07358 stdev: 0.02713
Best fitness: 0.14750 - size: (37, 59) - species 53 - id 12436
Average adjusted fitness: 0.050
Mean genetic distance 2.951, standard deviation 0.554
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 23.920 sec (23.495 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.07480 stdev: 0.02773
Best fitness: 0.14750 - size: (37, 59) - species 54 - id 12436
Average adjusted fitness: 0.056
Mean genetic distance 2.934, standard deviation 0.562
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 23.599 sec (23.492 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.07393 stdev: 0.02834
Best fitness: 0.14750 - size: (37, 59) - species 53 - id 12436
Average adjusted fitness: 0.060
Mean genetic distance 2.860, standard deviation 0.579
Population of 200 members in 10 species
Total extinctions: 0
Generation time: 23.705 sec (23.556 avera

...

Population's average fitness: 0.07422 stdev: 0.03079
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.075
Mean genetic distance 2.705, standard deviation 0.362
Population of 201 members in 12 species
Total extinctions: 0
Generation time: 28.241 sec (31.212 average)

 ****** Running generation 2903 ****** 

Population's average fitness: 0.07632 stdev: 0.03050
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.055
Mean genetic distance 2.734, standard deviation 0.342
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 28.083 sec (31.089 average)

 ****** Running generation 2904 ****** 

Population's average fitness: 0.08144 stdev: 0.03085
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.056
Mean genetic distance 2.750, standard deviation 0.340
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 29.592 sec 

Mean genetic distance 2.710, standard deviation 0.435
Population of 203 members in 10 species
Total extinctions: 0
Generation time: 29.072 sec (29.938 average)

 ****** Running generation 2926 ****** 

Population's average fitness: 0.08185 stdev: 0.03214
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.055
Mean genetic distance 2.708, standard deviation 0.429
Population of 202 members in 10 species
Total extinctions: 0
Generation time: 29.730 sec (29.865 average)

 ****** Running generation 2927 ****** 

Population's average fitness: 0.08068 stdev: 0.03314
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.059
Mean genetic distance 2.703, standard deviation 0.432
Population of 200 members in 10 species
Total extinctions: 0
Generation time: 31.686 sec (30.141 average)

 ****** Running generation 2928 ****** 

Population's average fitness: 0.08032 stdev: 0.03470
Best fitness: 0.18375 - size: (74, 114)

Population's average fitness: 0.08132 stdev: 0.03106
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.049
Mean genetic distance 2.743, standard deviation 0.376
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 36.465 sec (34.489 average)

 ****** Running generation 2950 ****** 

Population's average fitness: 0.08263 stdev: 0.03230
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.065
Mean genetic distance 2.715, standard deviation 0.395
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 36.249 sec (35.215 average)

 ****** Running generation 2951 ****** 

Population's average fitness: 0.07950 stdev: 0.03267
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.055
Mean genetic distance 2.709, standard deviation 0.401
Population of 194 members in 14 species
Total extinctions: 0
Generation time: 37.404 sec 

Mean genetic distance 2.687, standard deviation 0.412
Population of 202 members in 8 species
Total extinctions: 0
Generation time: 38.265 sec (33.171 average)

 ****** Running generation 2973 ****** 

Population's average fitness: 0.08016 stdev: 0.03169
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.048
Mean genetic distance 2.718, standard deviation 0.403
Population of 200 members in 9 species
Total extinctions: 0
Generation time: 40.005 sec (34.007 average)

 ****** Running generation 2974 ****** 

Population's average fitness: 0.07818 stdev: 0.03190
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.055
Mean genetic distance 2.711, standard deviation 0.420
Population of 201 members in 9 species
Total extinctions: 0
Generation time: 34.000 sec (34.251 average)

 ****** Running generation 2975 ****** 

Population's average fitness: 0.08227 stdev: 0.03220
Best fitness: 0.18375 - size: (74, 114) - 

Population's average fitness: 0.08367 stdev: 0.03220
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.057
Mean genetic distance 2.561, standard deviation 0.475
Population of 199 members in 5 species
Total extinctions: 0
Generation time: 30.465 sec (31.518 average)

 ****** Running generation 2997 ****** 

Population's average fitness: 0.08019 stdev: 0.03261
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.075
Mean genetic distance 2.568, standard deviation 0.456
Population of 199 members in 5 species
Total extinctions: 0
Generation time: 31.550 sec (31.297 average)

 ****** Running generation 2998 ****** 

Population's average fitness: 0.08109 stdev: 0.03368
Best fitness: 0.18375 - size: (74, 114) - species 683 - id 142759
Average adjusted fitness: 0.063
Mean genetic distance 2.502, standard deviation 0.456
Population of 200 members in 5 species
Total extinctions: 0
Generation time: 30.733 sec (31

In [42]:
numbers = [*range(26)] # number 2 > number 1
number_of_classification = len(numbers)


list_y = testy.tolist()
digits_indexes = []
for digit in numbers:
    li = [i for i in range(len(list_y)) if list_y[i] == digit]
    digits_indexes.extend(li)

samplesize = testX[digits_indexes].shape[0]
test_x_inputs =[tuple(c) for c in testX[digits_indexes].tolist()]
test_x_outputs = [tuple(c) for c in testy[digits_indexes].reshape(samplesize,1).tolist()]

In [43]:
outputs = []
for xi in test_x_inputs:
    output = winner_net.activate(xi)
    outputs.append(output)


px_outputs = softmax(np.array(outputs).reshape(samplesize, number_of_classification), axis=1)
# the index of maximum in each line
test_pred_outputs = np.argmax(px_outputs, axis = 1)
# replace index to real number, from higher number to lower number!!
for i in range(number_of_classification):
    index = number_of_classification - i - 1
    test_pred_outputs[test_pred_outputs==index] = numbers[index]

test_real_outputs = np.array(test_x_outputs).reshape(samplesize,)

In [44]:
np.sum(test_pred_outputs == test_real_outputs)/samplesize

0.14